In [1]:
# https://www.kaggle.com/datasets/trolukovich/apparel-images-dataset/data
# link al dataset 

In [2]:
#librerías instaladas en el entorno virtual

# !pip install tensorflow
# !pip install tensorflow-datasets

In [3]:
#librerías de python importadas

# generales 

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras


# preprocess
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils  import load_img
from tensorflow.keras.utils import img_to_array
import sklearn as sk
from sklearn.model_selection import train_test_split

# models
from keras.applications.vgg16 import VGG16
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
#import pickle

## Configuracion previa hardware

## Preprocessing CASIA 2.0 Image Tampering Detection Dataset

In [4]:
path_general = r"D:\1_data_science\trabajo_especializacion_datasets\Apparel_images_dataset"
list_dir_pg = os.listdir(path_general)
#armo diccionario de las clases y las imágenes
dicc_imgs = {}
for i in list_dir_pg:
    path = os.path.join(path_general, i)
    list_dir = os.listdir(path)
    dicc_imgs[i] = list_dir
#Me fijo los formatos únicos para cada clase, todos son jpg.
dicc_formatos = {}
for x in dicc_imgs.keys():
    formato = set([i.split(".")[-1] for i in dicc_imgs[x]])
    dicc_formatos[x] = formato

24

In [57]:
dicc_labels = {}
for i, clase in enumerate (dicc_imgs.keys()):
    dicc_labels[clase] = i+1

In [47]:
def cargar_imagen(img_path):
    try:
        # Cargar la imagen y redimensionarla
        img = load_img(img_path, target_size=(224, 224))
        # Convertir la imagen a un array
        img_array = np.array(img)
        # Reorganizar las dimensiones correctamente
        reshaped_img_array = img_array.reshape(1, 224, 224, 3)
        # Preprocesar la imagen
        preprocessed_img = preprocess_input(reshaped_img_array)
        return preprocessed_img
    except Exception as e:
        print(f"Error procesando la imagen {img_path}: {e}")
        return None

def dataset_builder(dataset_path):
    dataset = {}
    valid_formats = {".jpg", ".jpeg", ".tif", ".bmp", ".png"}
    
    # Itera por las carpetas de la ruta al dataset
    for class_folder in os.listdir(dataset_path):
        dataset[class_folder] = []
        class_path = os.path.join(dataset_path, class_folder)
        for img in os.listdir(class_path):
            img_path = os.path.join(class_path, img)
            _, img_format = os.path.splitext(img_path)
            if img_format.lower() in valid_formats:
                preprocessed_img = cargar_imagen(img_path)
                if preprocessed_img is not None:  # Validar si se procesó correctamente
                    dataset[class_folder].append(preprocessed_img)
                    
    return dataset
    

In [48]:
# Armo los datasets de imagenes autenticas y tampered
dicc_preproceded_imgs = dataset_builder(path_general)

In [55]:
# itero sobre las clases para saber cuantos elemenbtos hay por clase
conteos = {}
for key in dicc_preproceded_imgs.keys():
    conteos[key] = len(dicc_preproceded_imgs[key])
total = sum(conteos.values())
print (conteos, total)

{'black_dress': 450, 'black_pants': 871, 'black_shirt': 715, 'black_shoes': 766, 'black_shorts': 328, 'blue_dress': 502, 'blue_pants': 798, 'blue_shirt': 741, 'blue_shoes': 523, 'blue_shorts': 299, 'brown_pants': 311, 'brown_shoes': 464, 'brown_shorts': 40, 'green_pants': 227, 'green_shirt': 230, 'green_shoes': 455, 'green_shorts': 135, 'red_dress': 800, 'red_pants': 308, 'red_shoes': 610, 'white_dress': 818, 'white_pants': 274, 'white_shoes': 600, 'white_shorts': 120} 11385


In [62]:
for indice in dicc_preproceded_imgs.keys():
    X = []
    Y = []
    label = dicc_labels[indice]
    x = np.array(dicc_preproceded_imgs[indice])
    y = np.array([label for i in range(len(X))])
    X.append(x)
    Y.append(y)
print ("largo de X", len(X), "largo de Y", len(Y))

largo de X 1 largo de Y 1


In [53]:
tp_q = len(tp)
au_q = len(au)
tp_label = np.ones(tp_q)
au_label = np.zeros(au_q)

# Concatenar imágenes y etiquetas
X = np.concatenate((tp, au), axis=0)
y = np.concatenate((tp_label, au_label), axis=0)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_train, X_val, y_train , y_val = train_test_split(X_train, y_train, train_size=0.7, random_state=42, stratify=y_train)

In [54]:
print (X_train.shape, X_val.shape, X_test.shape)

(6154, 224, 224, 3) (2638, 224, 224, 3) (3768, 224, 224, 3)


## Definiendo la Arquitectura de la VGG16

In [55]:
model = VGG16()
categories = 2


In [56]:
#elijo las layers que conservo y las freezo para no modficarlas en el training

input_shape = (224, 224)

model = Model(inputs = model.inputs, outputs = model.layers[-2].output, input_shape=input_shape+(3,))
for layer in model.layers:
    layer.trainable = False

In [57]:
# creo nueva capa para clasificar y la agrego al modelo final
nueva_dense = tf.keras.layers.Dense (categories, activation = 'softmax')
model_final = Model(inputs = model.inputs, outputs = nueva_dense(model.output))

In [58]:
model_final.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,268,738 (512.19 MB)

 Trainable params: 8,194 (32.01 KB)

 Non-trainable params: 134,260,544 (512.16 MB)

In [59]:
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'], batch_size=32)

In [67]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

Epoch 1/10


c:\Users\Admin\.conda\envs\datascience\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_70']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 4096)